In [5]:
import pandas as pd

df = pd.read_csv("top50_posts_per_user_reduced.csv")       # autor, trait, post, similarity
labels = pd.read_csv("authors_train.csv")  # username + 5 rasgos

df = df.merge(labels, on="username")

df.head()




,username,trait,post,similarity,agreeableness,openness,conscientiousness,extraversion,neuroticism
0,-Areopagan-,Agreeableness,I am smarter than you and will work you into d...,0.476548,0.0,99.0,96.0,60.0,1.0
1,-Areopagan-,Agreeableness,"I have two friends. I alienate everyone, event...",0.472431,0.0,99.0,96.0,60.0,1.0
2,-Areopagan-,Agreeableness,Yeah I wouldnt want to deal with someone like ...,0.390348,0.0,99.0,96.0,60.0,1.0
3,-Areopagan-,Agreeableness,your first and second question is the same que...,0.328232,0.0,99.0,96.0,60.0,1.0
4,-Areopagan-,Openness,I am smarter than you and will work you into d...,0.357194,0.0,99.0,96.0,60.0,1.0


In [8]:
#esta funcion lo que busca es indicar cual es la columna objetivo en cada fila, en base al trait de la columna trait lo que coge como etiqueta es la columna que se llame igual al valor que hay en trait
def get_target(row):
    return str(row[row["trait"].lower()])  # valor numérico correcto

df["input_text"] = df.apply(lambda row: f"predict {row['trait']}: {row['post']}", axis=1)
df["target_text"] = df.apply(get_target, axis=1)

In [9]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["input_text", "target_text"]])
dataset = dataset.train_test_split(test_size=0.1)


In [10]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    enc = tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["target_text"],
            padding="max_length",
            truncation=True,
            max_length=10
        )
    enc["labels"] = labels["input_ids"]
    return enc

tokenized = dataset.map(preprocess, batched=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/139886 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/15543 [00:00<?, ? examples/s]

In [11]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-small")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
training_args = TrainingArguments(
    output_dir="./t5_personality",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    report_to="none",
    learning_rate=3e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
)


In [16]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [ ]:
def predict_trait(trait, text):
    input_text = f"predict {trait}: {text}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

print(predict_trait("extraversion", "I enjoy meeting new people"))
